In [1]:
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.agents.react import (
    REACT_INSTRUCTION_HOTPOTQA,
)
from agential.cog.prompts.benchmarks.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT

import dotenv

dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


In [3]:
agent = ReActAgent(llm=llm, mode={"qa": "hotpotqa"})

In [4]:
q = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

In [6]:
out = agent.generate(
    question=q,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_HOTPOTQA,
    additional_keys={},
    reset=True,
    max_steps=3
)

<PROMPT AGENT=======================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 3 steps.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny wa